In [3]:
import joblib
import mne
import numpy as np
import pandas as pd
from imblearn.under_sampling import RandomUnderSampler
from lightgbm import LGBMClassifier
from sklearn.metrics import (accuracy_score, average_precision_score,
                             confusion_matrix, f1_score, precision_score,
                             recall_score, roc_auc_score)
from sklearn.model_selection import StratifiedKFold

from add_subject_utils import get_subj_data, map_nan_index

In [ ]:
mne.set_log_level("error")

In [4]:
path = r"D:\Shaked_data\filtered_with_frontal\P54_frontal_filtered.fif"
raw = mne.io.read_raw(path)

Opening raw data file D:\Shaked_data\filtered_with_frontal\P54_frontal_filtered.fif...


d:\user\Desktop\Noa_Regev\Students\Shaked\spikes_notebooks\add_subject_utils.py:236: RuntimeWarning: This filename (D:\Shaked_data\filtered_with_frontal\P54_frontal_filtered.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  return mne.io.read_raw_fif(path)


Isotrak not found
    Range : 0 ... 28802218 =      0.000 ... 28802.218 secs
Ready.


In [4]:
raw.plot()

Using qt as 2D backend.


<mne_qt_browser._pg_figure.MNEQtBrowser(0x241cac23030) at 0x00000241CB6F0080>

In [1]:
path = r"F:\EGI cleaning\2.mff"
raw = mne.io.read_raw(path)
raw.crop(tmax=60*10)
raw.plot()

Reading EGI MFF Header from F:\EGI cleaning\2.mff...


C:\Users\user\AppData\Local\Temp\ipykernel_2988\4181147819.py:3: FutureWarning: events_as_annotations defaults to False in 1.8 but will change to True in 1.9, set it explicitly to avoid this warning
  raw = mne.io.read_raw(path)


    Reading events ...
    Assembling measurement info ...
    Excluding events {} ...
    Synthesizing trigger channel "STI 014" ...
Using qt as 2D backend.


Channels marked as bad:
none


Rest of add_new_subj notebook

In [ ]:
# extract scalp features
eog1 = get_subj_data(raw, 'EOG1', depth=False)
eog2 = get_subj_data(raw, 'EOG2', depth=False)
# combine and rename columns
subj_feat = pd.concat([eog1, eog2], axis=1, ignore_index=True) 
subj_feat.columns = [f'eog1_{col}' for col in eog1.columns] + [f'eog2_{col}' for col in eog2.columns]
# save to pkl
# subj_feat.to_pickle(rf'D:\Bonn\P{subj}_zeeg_feat.pkl')
subj_feat

,eog1_subj,eog1_epoch_id,eog1_app_entropy,eog1_decorr_time,eog1_energy_freq_bands_theta,eog1_energy_freq_bands_alpha,eog1_energy_freq_bands_sigma,eog1_energy_freq_bands_beta,eog1_energy_freq_bands_gamma,eog1_energy_freq_bands_fast,...,eog2_energy_freq_bands_st,eog2_energy_freq_bands_sg,eog2_energy_freq_bands_sf,eog2_energy_freq_bands_gt,eog2_energy_freq_bands_gf,eog2_energy_freq_bands_ft,eog2_chan_name,eog2_chan_ptp,eog2_chan_kurt,eog2_epoch
0,701,0,0.172145,-1.000,0.054619,0.034944,0.030562,0.090199,0.092207,0.000079,...,3.366357,0.614865,493.295773,5.474957,802.283626,0.006824,EOG2,10.359066,3.673379,"[0.2910601865218791, 0.27085540444514555, 0.24..."
1,701,1,0.173184,-1.000,0.046688,0.046029,0.047431,0.132119,0.072265,0.000273,...,0.695229,0.724523,370.600312,0.959568,511.509633,0.001876,EOG2,10.359066,3.673379,"[0.42728626838803074, 0.4326197366924741, 0.43..."
2,701,2,0.177867,-1.000,0.032664,0.026965,0.015006,0.096482,0.060310,0.000723,...,0.233752,0.288732,23.563429,0.809581,81.610027,0.009920,EOG2,10.359066,3.673379,"[-0.9316964981397581, -0.9154853175538413, -0...."
3,701,3,0.121215,-1.000,0.040300,0.053474,0.059047,0.072183,0.069587,0.000545,...,13.485761,0.625284,66.848866,21.567415,106.909595,0.201735,EOG2,10.359066,3.673379,"[-0.4146620600095086, -0.43864427108781995, -0..."
4,701,4,0.079268,-1.000,0.159191,0.051980,0.010390,0.062352,0.094975,0.000335,...,0.092975,0.211318,58.348508,0.439977,276.117657,0.001593,EOG2,10.359066,3.673379,"[-0.4385535960016858, -0.4189118817956746, -0...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1192,701,1192,0.081332,-1.000,0.144600,0.056448,0.013222,0.037078,0.017838,0.000049,...,0.093380,0.767438,387.699334,0.121677,505.186288,0.000241,EOG2,10.359066,3.673379,"[0.21704725781916898, 0.22380713474712743, 0.2..."
1193,701,1193,0.033331,-1.000,0.029814,0.013494,0.015413,0.069475,0.029345,0.000052,...,0.264601,0.315946,543.724512,0.837489,1720.940141,0.000487,EOG2,10.359066,3.673379,"[0.9136661454550974, 0.9162567031119532, 0.921..."
1194,701,1194,0.065752,-1.000,0.035587,0.026600,0.021840,0.067818,0.045333,0.000124,...,0.404642,0.648315,470.665064,0.624144,725.981701,0.000860,EOG2,10.359066,3.673379,"[-0.8245014135779527, -0.8289155812807444, -0...."
1195,701,1195,0.093725,0.048,0.030497,0.030434,0.026980,0.073301,0.047034,0.000111,...,0.356107,0.641986,618.843285,0.554696,963.951115,0.000575,EOG2,10.359066,3.673379,"[-1.6877949656409865, -1.6818209465981702, -1...."


In [ ]:
meta_data = ['subj', 'epoch_id', 'chan_name', 'epoch']
feat_to_choose_depth = ['teager_kaiser_energy_1_std', 'teager_kaiser_energy_2_std', 'chan_ptp', 'ptp_amp', 'hjorth_mobility', 'hjorth_complexity', 'teager_kaiser_energy_3_std', 'chan_kurt', 'teager_kaiser_energy_5_mean', 'teager_kaiser_energy_0_std', 'kurtosis', 'teager_kaiser_energy_0_mean', 'teager_kaiser_energy_1_mean', 'teager_kaiser_energy_5_std', 'samp_entropy']
feat_to_choose2 = ['chan', 'bands_gf', 'bands_gamma', 'bands_fast', 'bands_beta/gamma', 'bands_theta/fast', 'ptp', 'skew', 'teager', 'bands_sf', 'bands_bf', 'bands_bg', 'rms', 'katz', 'kurt', 'slope', 'mobility', 'hurst', 'wavelet']
feat_to_choose3 = ['chan', 'bands_gf', 'bands_gamma', 'energy_freq_bands_fast', 'bands_beta/gamma', 'bands_theta/fast', 'ptp', 'skew', 'teager', 'bands_sf', 'bands_bf', 'rms', 'kurt', 'slope', 'mobility', 'eog1_wavelet_coef_energy_0']
remove_3 = ['gamma/beta', 'fast/theta', 'fast/gamma']
remove_4 = ['gamma/sigma', 'fast/alpha', 'coef_1', '2_mean']
# get only columns that contain the string in feat_to_choose
x_feat = subj_feat[subj_feat.columns[subj_feat.columns.str.contains('|'.join(feat_to_choose2))]]
# now remove the metadata and some unwanted features
clean_feat = x_feat[x_feat.columns[~x_feat.columns.str.contains('|'.join(meta_data+remove_3))]]
clean_feat

,eog1_energy_freq_bands_gamma,eog1_energy_freq_bands_fast,eog1_hjorth_mobility,eog1_hjorth_mobility_spect,eog1_hurst_exp,eog1_katz_fd,eog1_kurtosis,eog1_pow_freq_bands_gamma,eog1_pow_freq_bands_fast,eog1_pow_freq_bands_theta/fast,...,eog2_wavelet_coef_energy_1,eog2_wavelet_coef_energy_2,eog2_wavelet_coef_energy_3,eog2_wavelet_coef_energy_4,eog2_energy_freq_bands_bg,eog2_energy_freq_bands_bf,eog2_energy_freq_bands_sf,eog2_energy_freq_bands_gf,eog2_chan_ptp,eog2_chan_kurt
0,0.092207,0.000079,0.072972,1.724620,0.752238,1.242145,1.728290,0.017627,0.000323,1544.998765,...,0.001134,0.048922,0.629345,1.829672,0.694779,557.409740,493.295773,802.283626,10.359066,3.673379
1,0.072265,0.000273,0.111870,2.107009,0.618507,1.455004,3.690966,0.041697,0.001513,314.222095,...,0.000157,0.016922,0.146240,2.472885,2.287288,1169.969886,370.600312,511.509633,10.359066,3.673379
2,0.060310,0.000723,0.241754,1.348111,0.835812,1.202516,2.373995,0.023218,0.000578,829.726495,...,0.000729,0.039661,0.212950,0.856875,1.336773,109.094089,23.563429,81.610027,10.359066,3.673379
3,0.069587,0.000545,0.147197,1.220273,0.871311,1.220477,1.959333,0.008369,0.000225,2373.250111,...,0.001720,0.042194,0.204139,1.383887,0.794851,84.977251,66.848866,106.909595,10.359066,3.673379
4,0.094975,0.000335,0.119188,3.409393,0.819717,1.303538,1.565871,0.005502,0.000176,3013.886212,...,0.001212,0.031702,0.528629,0.481083,0.609592,168.319031,58.348508,276.117657,10.359066,3.673379
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1192,0.017838,0.000049,0.160558,1.170072,1.252324,1.255017,2.791343,0.004847,0.000779,674.446268,...,0.000161,0.004853,0.106114,1.726922,2.168267,1095.378631,387.699334,505.186288,10.359066,3.673379
1193,0.029345,0.000052,0.165910,2.281380,0.989556,1.081192,1.772199,0.006619,0.001344,390.683242,...,0.000052,0.007928,0.155167,0.547002,2.103573,3620.123816,543.724512,1720.940141,10.359066,3.673379
1194,0.045333,0.000124,0.027994,1.724609,0.775838,1.194464,1.403741,0.003874,0.000345,1539.859461,...,0.000088,0.004882,0.123664,1.340190,1.950196,1415.806745,470.665064,725.981701,10.359066,3.673379
1195,0.047034,0.000111,0.100966,1.415740,0.810222,1.170506,2.907787,0.014566,0.001176,434.453827,...,0.000136,0.006415,0.164723,1.440781,1.232480,1188.050163,618.843285,963.951115,10.359066,3.673379


In [ ]:
# unbalanced model
metrics = {'accuracy': [], 'precision': [], 'sensitivity': [], 'specificity': [],'f1': [], 'ROCAUC': [], 'PRAUC': []}
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=8)
i = 1
x, y = clean_feat, y_depth
for train_index, test_index in kf.split(x, y):
    print(f'Fold {i}')
    i += 1
    z_model = LGBMClassifier()
    x_train_fold, x_test_fold = x.iloc[train_index], x.iloc[test_index]
    y_train_fold, y_test_fold = y[train_index], y[test_index]
    z_model.fit(x_train_fold, y_train_fold)
    y_pred = z_model.predict(x_test_fold)
    y_true = y_test_fold
    # save scores in dict
    metrics['accuracy'].append(accuracy_score(y_true, y_pred))
    metrics['precision'].append(precision_score(y_true, y_pred))
    metrics['sensitivity'].append(recall_score(y_true, y_pred))
    metrics['f1'].append(f1_score(y_true, y_pred))
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    metrics['specificity'].append(tn / (tn + fp))
    metrics['ROCAUC'].append(roc_auc_score(y_true, y_pred))
    metrics['PRAUC'].append(average_precision_score(y_true, y_pred))

# print results as df
results = pd.DataFrame(metrics)
# add mean row
results.loc['mean'] = results.mean()
print(sum(y)/len(y))
results

Fold 1
[LightGBM] [Info] Number of positive: 68, number of negative: 889
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001462 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 23460
[LightGBM] [Info] Number of data points in the train set: 957, number of used features: 92
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.071055 -> initscore=-2.570590
[LightGBM] [Info] Start training from score -2.570590
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Fold 2
[LightGBM] [Info] Number of positive: 68, number of negative: 889
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001084 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 23460
[LightGBM] [Info] Number of data points in the train set: 957, number of used feature

,accuracy,precision,sensitivity,specificity,f1,ROCAUC,PRAUC
0,0.925000,0.000000,0.000000,0.995516,0.000000,0.497758,0.070833
1,0.933333,0.666667,0.117647,0.995516,0.200000,0.556581,0.140931
2,0.937238,1.000000,0.117647,1.000000,0.210526,0.558824,0.180409
3,0.945607,0.833333,0.294118,0.995495,0.434783,0.644807,0.295307
4,0.937238,0.625000,0.294118,0.986486,0.400000,0.640302,0.234033
mean,0.935683,0.625000,0.164706,0.994603,0.249062,0.579654,0.184303


In [ ]:
# undersample
rus = RandomUnderSampler(random_state=8)
x, y = rus.fit_resample(clean_feat, y_depth)
len(y)

170

In [ ]:
metrics = {'accuracy': [], 'precision': [], 'sensitivity': [], 'specificity': [],'f1': [], 'ROCAUC': [], 'PRAUC': []}
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=8)
i = 1
for train_index, test_index in kf.split(x, y):
    print(f'Fold {i}')
    i += 1
    z_model = LGBMClassifier()
    x_train_fold, x_test_fold = x.iloc[train_index], x.iloc[test_index]
    y_train_fold, y_test_fold = y[train_index], y[test_index]
    z_model.fit(x_train_fold, y_train_fold)
    y_pred = z_model.predict(x_test_fold)
    y_true = y_test_fold
    # save scores in dict
    metrics['accuracy'].append(accuracy_score(y_true, y_pred))
    metrics['precision'].append(precision_score(y_true, y_pred))
    metrics['sensitivity'].append(recall_score(y_true, y_pred))
    metrics['f1'].append(f1_score(y_true, y_pred))
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    metrics['specificity'].append(tn / (tn + fp))
    metrics['ROCAUC'].append(roc_auc_score(y_true, y_pred))
    metrics['PRAUC'].append(average_precision_score(y_true, y_pred))

# print results as df
results = pd.DataFrame(metrics)
# add mean row
results.loc['mean'] = results.mean()
print(sum(y)/len(y))
results

In [ ]:
# a new model includes all the data
z_model = LGBMClassifier().fit(x, y)
y_proba = z_model.predict_proba(clean_feat).T
y_scalp = [p > 0.8 for p in y_proba[1]]
print(sum(y_scalp), sum(y_depth))
index_map = map_nan_index(r'D:\Bonn\012fn1\P701_mtl_clean.edf')
scalp_indexes = np.where(np.array(y_scalp) == True)[0]
scalp_onsets = [index_map[int(x)] / 4 for x in scalp_indexes]
depth_indexes = np.where(y_depth == 1)[0]
depth_onsets = [index_map[int(x)] / 4 for x in depth_indexes]
both = [x for x in scalp_onsets if x in depth_onsets]
depth_without_both = [x for x in depth_onsets if x not in both]
scalp_without_both = [x for x in scalp_onsets if x not in both]
# all annot
annot = mne.Annotations(scalp_without_both, [0.25] * len(scalp_without_both), ['scalp'] * len(scalp_without_both)).append(depth_without_both, [0.25] * len(depth_without_both), ['depth'] * len(depth_without_both)).append(both, [0.25] * len(both), ['both'] * len(both))
raw.set_annotations(annot)

raw.plot(duration=30, scalings='auto')

In [ ]:
v2_model = joblib.load('validation_models_v2\\lgbm_s13_f98_b_sym.pkl')